# Running the fleet of Virtual Wind Turbines and Edge Devices

**SageMaker Studio Kernel**: Data Science

After deploying the inference app on the 5 IoT Core devices(EC2 instances, in this case), its time to set up the virtual fleet of turbines and the simulator app along with it. In this exercise you will run a local application written in Python3 that simulates 5 Wind Turbines. This application will interact with IoT Core devices via IoT MQTT to exchange information.

Here you'll be the **Wind Turbine Farm Operator**. It's possible to visualize the data flowing from the sensors to the ML Model and analyze the anomalies. Also, you'll be able to inject noise (pressing some buttons) in the data to simulate potential anomalies with the equipment.

<table border="0" cellpading="0">
    <tr>
        <td align="center"><b>STEP-BY-STEP</b></td>
        <td align="center"><b>APPLICATION ARCHITECTURE</b></td>
    </tr>
    <tr>
        <td><img src="../../../imgs/EdgeManagerWorkshop_Macro.png" width="500px"></img></td>
        <td><img src="../../../imgs/ggv2_lab4_app_arch.png"  width="500px"></img></td>
    </tr>
</table>  

The components of the Simulator are:
 - [Simulator](simulator/simulator.py): Program that launches the virtual wind turbines and the edge devices. It also maintains a simple dashboard to show the data which is being sent for anomaly detection to the detector.
 - [Turbine](simulator/turbine.py): Virtual Wind Turbine. It reads the raw data collected from the 3D Prited Mini Turbine and stream it as a circular buffer. It also has a graphical representation in **IPython Widgets** that is rendered by the Simulator/Dashboard. It contains all the business logic of maintaining the virtual turbine.
 - [MQTT Client](simulator/mqttclient.py): An abstraction layer for MQTT communication providing convenience methods to publish and subscribe to topics.

First, let us restore variables stored in the previous exercise

In [ ]:
%store -r project_id
%store -r deployment_id

### Install some dependencies

In [ ]:
!pip install ipywidgets
!pip install awsiotsdk

### Download the sensors data (streamed by the virtual wind turbines)

The below cell checks if the data is already downloaded.
#  

 if not, then it downloads the data. Otherwise it skips the downloading.

In [ ]:
import os.path

if os.path.isfile('../../data/dataset_wind.csv.gz') is not True:
    print('file not present, downloading now..')
    os.system("mkdir -p ../../data")
    os.system("curl https://aws-ml-blog.s3.amazonaws.com/artifacts/monitor-manage-anomaly-detection-model-wind-turbine-fleet-sagemaker-neo/dataset_wind_turbine.csv.gz -o ../../data/dataset_wind.csv.gz")
    print('download complete !!')
else:
    print('file already present, no need to download again !')

In [ ]:
project_name='<<PROJECT_NAME>>'

import boto3

sm_client = boto3.client('sagemaker')
project_id = sm_client.describe_project(ProjectName=project_name)['ProjectId']
bucket_name = 'sagemaker-wind-turbine-farm-%s' % project_id

# Simulating The Wind Turbine Farm
Now its time to run our simulator and start playing with the turbines and with the anomalies
 > After clicking on **Start**, each turbine will start buffering some data. It takes a few seconds but after completing this process, the application runs in real-time   
 > Try to press some buttons while the simulation is running, to inject noise in the data and see some anomalies  


In [ ]:
import os
import sys
sys.path.insert(0, os.path.join(os.path.realpath('../../'),'simulator'))

In [ ]:
import turbine
import simulator
import mqttclient
from importlib import reload
import logging

logging.basicConfig(level=logging.INFO)

reload(mqttclient)
reload(turbine)
reload(simulator)

simulator = simulator.WindTurbineFarmSimulator(5)

In [ ]:
simulator.start()
simulator.show()

Once your experiment is finished, stop the simulator. This will in turn also stops/halts all the turbines.

In [ ]:
simulator.halt()

 > If you want to experiment with the deployment process, with the wind farm running, go back to [Exercise #2](02-Training-with-Pytorch.ipynb#Building-the-Deployment-Package-SageMaker-Edge-Manager) (in the last cells), go back to lab02 notebook and deploy a new `version` of the model, create a new greengrass component version of the model and create a new deployment version deploying this new model version.

## Cleanup
Run the next cell only if you already finished exploring/hacking the content of the workshop.  
This code will delete all the resouces created so far, including the **SageMaker Project** you've created

In [ ]:
import boto3
import time
from shutil import rmtree

iot_client = boto3.client('iot')
sm_client = boto3.client('sagemaker')
s3_resource = boto3.resource('s3')
ggv2_client = boto3.client('greengrassv2')

policy_name='WindTurbineFarmPolicy-%s' % project_id
thing_group_name='WindTurbineFarm-%s' % project_id
fleet_name='wind-turbine-farm-%s' % project_id

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


In [ ]:
#Cancel the deployment of greengrass v2
ggv2_client.cancel_deployment(deploymentId=deployment_id)

In [ ]:
#Delete the core devices:
for i in range(5):
    ggv2_client.delete_core_device(coreDeviceThingName='edge-device-'+str(i))

In [ ]:
# Delete all files from the S3 Bucket
s3_resource.Bucket(bucket_name).objects.all().delete()

In [ ]:
# delete the ggv2 components
ggv2_client.delete_component(arn='arn:aws:greengrass:{}:{}:components:aws.samples.windturbine.detector:versions:1.0.0'.format(region,account_id))
ggv2_client.delete_component(arn='arn:aws:greengrass:{}:{}:components:aws.samples.windturbine.model:versions:1.0.0'.format(region,account_id))

In [ ]:
# now deregister the devices from the fleet
resp = sm_client.list_devices(DeviceFleetName=fleet_name)
devices = [d['DeviceName'] for d in resp['DeviceSummaries']]
if len(devices) > 0:
    sm_client.deregister_devices(DeviceFleetName=fleet_name, DeviceNames=devices)

In [ ]:
# now deregister the devices from the fleet
for i,cert_arn in enumerate(iot_client.list_targets_for_policy(policyName=policy_name)['targets']):
    for t in iot_client.list_principal_things(principal=cert_arn)['things']:
        iot_client.detach_thing_principal(thingName=t, principal=cert_arn)
    iot_client.detach_policy(policyName=policy_name, target=cert_arn)
    certificateId = cert_arn.split('/')[-1]
    iot_client.update_certificate(certificateId=certificateId, newStatus='INACTIVE')
    iot_client.delete_certificate(certificateId=certificateId)

In [ ]:
iot_client.delete_role_alias(roleAlias='SageMakerEdge-%s' % fleet_name)

In [ ]:
iot_client.delete_thing_group(thingGroupName=thing_group_name)

In [ ]:
if os.path.isdir('agent'): rmtree('agent')
sm_client.delete_project(ProjectName=project_name)

Mission Complete!  
Congratulations! :)